In [16]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [3]:
data_dir = '../capstone/imagedata' # call data directory
classes = os.listdir(data_dir)

height = 128 # image height
width = 128 # image width

with open('workout_label.txt', 'w') as f:
    for workout_class in classes:
        f.write(f'{workout_class}\n')

data = []
labels = []   

for dirname, _, filenames in os.walk(data_dir):
    data_class = dirname.split(os.path.sep)[-1]
    for filename in filenames:
        img_path = os.path.join(dirname, filename)
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (height , width))
        
        data.append(image)
        labels.append(classes.index(data_class))
        
data = np.array(data)
labels = np.array(labels)

labels = to_categorical(labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)

In [5]:
num_training_images = len(X_train)
print("Total training images:", num_training_images)

Total training images: 12467


In [6]:
num_test_images = len(X_test)
print("Total test images:", num_test_images)

Total test images: 1386


In [7]:
train_data = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_data.fit(X_train)

test_data = ImageDataGenerator(rescale = 1./255)

test_data.fit(X_test)

In [40]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(width, height, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(classes), activation = 'softmax'))
    model.summary()
    return model

In [41]:
workout_model = create_model()
workout_model

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 62, 62, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 31, 64)       0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 31, 31, 64)      

In [44]:
workout_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
workout_model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test))


Epoch 1/30
98/98 [==============================] - 136s 1s/step - loss: 0.7050 - accuracy: 0.7942 - val_loss: 0.3764 - val_accuracy: 0.9170
Epoch 2/30
98/98 [==============================] - 146s 1s/step - loss: 0.6409 - accuracy: 0.8093 - val_loss: 0.3738 - val_accuracy: 0.9250
Epoch 3/30
98/98 [==============================] - 139s 1s/step - loss: 0.5405 - accuracy: 0.8309 - val_loss: 0.3180 - val_accuracy: 0.9307
Epoch 4/30
98/98 [==============================] - 137s 1s/step - loss: 0.4643 - accuracy: 0.8491 - val_loss: 0.2964 - val_accuracy: 0.9358
Epoch 5/30
98/98 [==============================] - 160s 2s/step - loss: 0.4458 - accuracy: 0.8624 - val_loss: 0.2795 - val_accuracy: 0.9408
Epoch 6/30
98/98 [==============================] - 161s 2s/step - loss: 0.4246 - accuracy: 0.8675 - val_loss: 0.2949 - val_accuracy: 0.9343
Epoch 7/30
98/98 [==============================] - 158s 2s/step - loss: 0.3941 - accuracy: 0.8743 - val_loss: 0.2989 - val_accuracy: 0.9430
Epoch 8/30
98

In [45]:
loss, accuracy = workout_model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

44/44 [==============================] - 2s 56ms/step - loss: 0.2673 - accuracy: 0.9495
Loss:  0.26725471019744873
Accuracy:  0.9494949579238892


In [46]:
workout_model.save('workout_model')

converter = tf.lite.TFLiteConverter.from_saved_model('./workout_model')
tflite_model = converter.convert()

with open('workout_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: workout_model\assets


INFO:tensorflow:Assets written to: workout_model\assets
